In [27]:
import json,os
import numpy as np
import pandas as pd

### Cargo datos

In [28]:
ligue2team2rank = json.load( open("../../data/league2team2rank.json", "r") )

In [29]:
matches = pd.read_csv("../../data/matches.csv")
matches.head(2)

,match,home,away,winner,score_home,score_away,label,liga
0,2500089,1646,1659,1659,1,2,"Burnley - AFC Bournemouth, 1 - 2",England
1,2500090,1628,1627,1628,2,0,"Crystal Palace - West Bromwich Albion, 2 - 0",England


In [30]:
metrics = pd.read_csv("../../data/metrics.csv")
metrics.head(2)

,match,team1,direct_play,counterattack,T_maintenance,T_build_up,T_zona_media,flow_rate,crossing,pressure_point,pressure_loss,shots,liga
0,2499719,1609,1.046492,0.137967,0.008011,1.641360,0.404629,2.365172,0.475739,0.553544,-0.999813,2.012367,England
1,2499719,1631,-1.615181,-1.387848,-1.442053,0.599849,-1.344791,-0.581150,-0.981960,-1.274967,-0.617518,-2.056307,England


### Armo redes

Primero cruzo los datos por match

In [31]:
df0 = pd.merge(metrics, matches[['match', 'home', 'away', 'winner', 'score_home','score_away']], on='match', how='left')

df0.head(2)

,match,team1,direct_play,counterattack,T_maintenance,T_build_up,T_zona_media,flow_rate,crossing,pressure_point,pressure_loss,shots,liga,home,away,winner,score_home,score_away
0,2499719,1609,1.046492,0.137967,0.008011,1.641360,0.404629,2.365172,0.475739,0.553544,-0.999813,2.012367,England,1609,1631,1609,4,3
1,2499719,1631,-1.615181,-1.387848,-1.442053,0.599849,-1.344791,-0.581150,-0.981960,-1.274967,-0.617518,-2.056307,England,1609,1631,1609,4,3


In [32]:
metricas = ['direct_play','counterattack','T_maintenance','T_build_up',
            'T_zona_media','flow_rate','crossing','pressure_point','pressure_loss',	
            'shots']

ligas = ['England','France','Germany','Italy','Spain']


for liga in ligas:

    print (liga)

    carpeta = f'{liga}'
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)


    team2rank = ligue2team2rank[liga]
    df = df0[ df0['liga']==liga ]


    for metrica in metricas:

        print (f"\t{metrica}")

        d={}
        d2={}

        def f(row):

            # tomo datos
            team1 = str(row.team1)
            nodo1 = team2rank[team1]

            if row.team1==row.home:
                team2=str(row.away)
            else:
                team2=str(row.home)

            nodo2 = team2rank[team2]

            w = row[metrica] 

            try:
                d[(nodo1,nodo2)]+=w
            except:
                d[(nodo1,nodo2)]=w

            try:
                d2[(team1,team2)].append(w)
            except:
                d2[(team1,team2)]=[w]
            

        df.agg( lambda x: f(x), axis=1 )


        # armo red
        Nmax = max(elemento for tupla in d for elemento in tupla)
        red = []

        for i in range(1,Nmax):

            for j in range(i+1, Nmax+1):

                fij = d[(j,i)] - d[(i,j)] 

                red.append( [i,j, fij] )



        #guardo data
        arx_out = f'{liga}/red_{metrica}.dat'

        np.savetxt(
            arx_out,               
            red,               
            fmt='%d,%d,%.4e',         
            delimiter=',', 
            header='n1,n2,fij',
            comments=''
        )



England
	direct_play
	counterattack
	T_maintenance
	T_build_up
	T_zona_media
	flow_rate
	crossing
	pressure_point
	pressure_loss
	shots
France
	direct_play
	counterattack
	T_maintenance
	T_build_up
	T_zona_media
	flow_rate
	crossing
	pressure_point
	pressure_loss
	shots
Germany
	direct_play
	counterattack
	T_maintenance
	T_build_up
	T_zona_media
	flow_rate
	crossing
	pressure_point
	pressure_loss
	shots
Italy
	direct_play
	counterattack
	T_maintenance
	T_build_up
	T_zona_media
	flow_rate
	crossing
	pressure_point
	pressure_loss
	shots
Spain
	direct_play
	counterattack
	T_maintenance
	T_build_up
	T_zona_media
	flow_rate
	crossing
	pressure_point
	pressure_loss
	shots


In [33]:
d2

{('682', '695'): [-0.1150563424940563, -0.1150563424940563],
 ('695', '682'): [0.3932785855686542, -0.831514183629255],
 ('687', '692'): [-2.0563069528271645, -0.831514183629255],
 ('692', '687'): [-2.0563069528271645, -0.1150563424940563],
 ('678', '698'): [0.3932785855686542, -0.1150563424940563],
 ('698', '678'): [0.787573788388354, -0.1150563424940563],
 ('679', '756'): [-2.0563069528271645, -0.831514183629255],
 ('756', '679'): [-0.1150563424940563, -2.0563069528271645],
 ('680', '691'): [-2.0563069528271645, -2.0563069528271645],
 ('691', '680'): [0.3932785855686542, -0.831514183629255],
 ('683', '701'): [-0.1150563424940563, -0.1150563424940563],
 ('701', '683'): [-0.831514183629255, -0.831514183629255],
 ('676', '684'): [0.3932785855686542, 0.787573788388354],
 ('684', '676'): [-2.0563069528271645, -0.1150563424940563],
 ('675', '677'): [-0.1150563424940563, 1.8261942678390524],
 ('677', '675'): [0.3932785855686542, 0.3932785855686542],
 ('674', '714'): [0.3932785855686542, -0.